# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании. Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Дополнительно требуется обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

Для начала, импортируем все необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

Загрузим данные и выведем их основные параметры.

In [3]:
data = pd.read_csv('insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В данных пропусков нет. Проверим есть ли дубликаты:

In [4]:
data.duplicated().sum()

153

Дубликаты есть, избавимся от них.

In [5]:
data = data.drop_duplicates()
data.shape

(4847, 5)

Названия столбцов лучше заменим на английские:

In [6]:
data = data.rename({'Пол': 'gender', 
                    'Возраст': 'age',
                    'Зарплата': 'salary',
                    'Члены семьи': 'family',
                    'Страховые выплаты': 'payments'}, axis=1)

In [7]:
data.head()

,gender,age,salary,family,payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**  
Считаем, что предсказания не изменятся.

**Обоснование:**  
Чтобы доказать то, что предсказания не изменятся, нам нужно проверить равны ли предсказания до преобразования ($a$) и предсказания после преобразования ($a_1$):  
$$a = a_1$$

1. Подставим в формулу обучения преобразованную матрицу (она получается путём умножения исходных данных $X$ на неизвестную обратимую матрицу $P$). Таким образом, мы получим формулу для весов регрессии после преобразования данных ($w_1$): 
$$w_1 = ((XP)^T (XP))^{-1} (XP)^T y$$ 
От скобок для ($XP$) можем избавиться по свойству матриц $A(BC) = (AB)C = ABC$: 
$$(XP)^T (XP) = ((XP)^T X) P = (XP)^T XP$$ 
Получаем выражение: 
$$w_1 = ((XP)^T XP)^{-1}(XP)^T y$$  
  
2. Раскроем скобки для $(XP)^T$ по свойству $(AB)^T = B^T A^T$: 
$$w_1 = (P^T X^T XP)^{-1}P^T X^T y$$  
  
3. Сгрупируем $X^T$ и $X$ и раскроем скобки по свойству $(AB)^{-1} = B^{-1}A^{-1}$: 
$$w_1 = (P^T (X^T X)P)^{-1}P^T X^T y$$ 
$$w_1 = ((P^T (X^T X))P)^{-1}P^T X^T y$$ 
$$w_1 = P^{-1}(P^T (X^T X))^{-1}P^T X^T y$$ 
$$w_1 = P^{-1}(X^T X)^{-1}(P^T)^{-1}P^T X^T y$$ 

4. $(P^T)^{-1}P^T = E$, поэтому от этих матриц можем избавиться: 
$$w_1 = P^{-1}(X^T X)^{-1} X^T y$$ 
5. В получившееся выражение можем подставить формулу для весов изначальных данных. Получим выражение: 
$$w_1 = P^{-1}w$$
   
6. Так как мы получили формулу весов для преобразованных данных, можем подставить получившееся выражение в формулу для $a_1$: 
$$a_1 = XPP^{-1}w$$
$$a_1 = XEw$$
$$a_1 = Xw$$
$$a_1 = a$$  
  
Таким образом, мы доказали, что после преобразования исходных данных, предсказания не изменятся. Также, получили взаимосвязь параметров до и после преобразования: чтобы получить $w_1$ нужно умножить $w$ на обратную матрицу к $P$.

## Алгоритм преобразования

**Алгоритм**  
Напишем алгоритм и найдём разницу между предсказаниями до и после преобразования.

1. Создадим случайную обратимую матрицу размером 4х4 (так как у нас 4 признака в данных):

In [8]:
p_matrix = np.random.randint(100, size=(4, 4))
print('Обратимая матрица P')
print(p_matrix, '\n')

inverted_p_matrix = np.linalg.inv(p_matrix)
print('Матрица, обратная P')
print(inverted_p_matrix)

Обратимая матрица P
[[35 25 52 27]
 [58 79 81  8]
 [24 23 56 75]
 [57 13 69 27]] 

Матрица, обратная P
[[-0.68679263  0.13521506  0.14641399  0.24002338]
 [-0.18681562  0.05121094  0.04626761  0.04312087]
 [ 0.69809708 -0.1388353  -0.15681744 -0.2213567 ]
 [-0.24418205  0.04469018  0.06938248  0.07524845]]


2. Возьмём из данных признаки и целевой признак:

In [9]:
target = data['payments']
features = data.drop('payments', axis=1)

3. Применим масштабирование к признакам, так как они все находятся в разном масштабе:

In [10]:
scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

4. Рассчитаем вектор весов $w$ для исходных данных по формуле $w = (X^T X)^{-1} X^T y$

In [12]:
w = np.linalg.inv(features_scaled.T @ features_scaled) @ features_scaled.T @ target

5. После этого, вычислим вектор предсказаний $a = Xw$

In [13]:
a = features_scaled @ w

6. Повторим все предыдущие пункты для расчёта предсказаний по преобразованным данным ($a_1$). Для этого исходная матрица будет умножаться на матрицу $P$.

In [14]:
features_converted = features_scaled @ p_matrix
w_1 = np.linalg.inv(features_converted.T @ features_converted) @ features_converted.T @ target
a_1 = features_converted @ w_1

7. Посчитаем среднюю разницу между предсказаниями до и после преобразования:

In [15]:
abs(a_1 - a).mean()

9.66778193173157e-13

**Обоснование**

Разница между предсказаниями до и после преобразования минимальна и практически равна нулю. Алгоритм работает.

## Проверка алгоритма

Проверим алгоритм преобразования. Посчитаем метрику R2 для линейной регрессии до и после преобразований.

In [16]:
model = LinearRegression()
model.fit(features_scaled, target)
preds = model.predict(features_scaled)

print('R2 модели на исходных данных:', r2_score(target, preds))

R2 модели на исходных данных: 0.4302010044852068


In [17]:
model = LinearRegression()
model.fit(features_converted, target)
preds_converted = model.predict(features_converted)

print('R2 модели на преобразованных данных:', r2_score(target, preds_converted))

R2 модели на преобразованных данных: 0.4302010044852068


Метрики практически идентичны, алгоритм защиты персональных данных работает без потери качества предсказаний модели.